# Importing Packages

In [133]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import pickle

%reload_ext autoreload
%autoreload 2
from utils import *

import smote_variants as sv
import imbalanced_databases as imbd
import xgboost as xgb

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict, GridSearchCV,RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, classification_report, plot_confusion_matrix, mean_squared_error, confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import resample
from sklearn.dummy import DummyClassifier

from imblearn.under_sampling import CondensedNearestNeighbour, NearMiss, OneSidedSelection, NeighbourhoodCleaningRule, RandomUnderSampler, TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, SVMSMOTE
from imblearn.ensemble import BalancedBaggingClassifier, BalancedRandomForestClassifier, EasyEnsembleClassifier, RUSBoostClassifier
from imblearn.metrics import geometric_mean_score

from xgboost.sklearn import XGBClassifier

2021-06-14 07:29:33,658:DEBUG:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


# Importing Training and Validation Datasets

In [69]:
pickle_in = open("data/training_cleaned.pickle", "rb")
train = pickle.load(pickle_in)
pickle_in = open("data/validate_cleaned.pickle", "rb")
validate = pickle.load(pickle_in)

In [70]:
X_train = train.drop(["default"], axis=1)
y_train = train["default"]
X_valid = validate.drop(["default"], axis=1)
y_valid = validate["default"]

In [71]:
X_train.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,...,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6
0,1790.26,2,2,1,44,0,0,0,0,0,...,1278.35,800.60,847.12,981.81,107.99,179.13,107.42,107.42,179.03,33.08
1,5728.83,2,3,1,46,-1,-1,-1,0,-1,...,173.87,147.77,143.04,30.15,83.89,173.87,35.81,143.04,30.15,942.14
2,3580.52,2,2,1,47,-1,-1,-1,-1,-1,...,0.00,224.50,-14.18,-14.18,238.68,0.00,224.50,0.00,0.00,0.00
3,6086.88,2,2,1,29,0,0,0,0,0,...,2267.08,2288.06,1557.71,1575.25,80.02,89.26,92.56,60.26,68.07,75.58
4,5370.78,2,1,2,33,-2,-2,-2,-2,-2,...,1170.90,1198.01,995.38,80.96,966.99,1171.37,1198.58,995.67,80.96,6067.73


In [72]:
X_valid.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,...,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6
0,1074.16,1,2,2,25,0,0,0,0,0,...,414.66,450.43,491.10,530.92,53.71,71.61,53.71,53.71,53.71,71.61
1,5370.78,2,1,2,26,0,0,0,0,0,...,4177.89,3637.13,2783.53,2766.45,160.62,151.64,113.18,94.78,95.56,95.56
2,2506.36,2,3,1,32,0,0,0,0,0,...,2453.73,2497.52,2510.34,2513.96,87.04,111.43,107.42,87.29,89.51,91.45
3,4654.68,1,3,2,49,0,0,0,0,0,...,579.04,605.04,402.31,248.63,57.65,64.74,251.14,0.97,251.03,157.83
4,1790.26,2,2,2,36,0,0,0,0,0,...,1516.74,700.85,726.67,696.02,71.61,53.71,35.81,64.45,0.00,35.81


# Standardize Datasets

In [73]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_ss = scaler.transform(X_train)
X_valid_ss = scaler.transform(X_valid)

# Importing Model

In [74]:
gbc = GradientBoostingClassifier(random_state=42).fit(X_train_ss, y_train)
y_pred = gbc.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred, gbc)

Accuracy Score:  0.821
F1 Score:  0.4688427299703264
ROC-AUC Score:  0.7808000154695354
Recall Score:  0.361005331302361
Precision Score:  0.6685472496473907
PR-AUC Score:  0.5454979238062434


In [82]:
params = {'n_estimators': range(20, 81, 10)}
gs_gbc = GridSearchCV(estimator = GradientBoostingClassifier(random_state=42),
                      param_grid = params, 
                      n_jobs = 4,
                      iid = False, 
                      cv = 5, 
                      scoring = 'roc_auc',
                      verbose = 1).fit(X_train_ss, y_train)
print("Best: %f using %s" % (gs_gbc.best_score_, gs_gbc.best_params_))

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:   27.1s finished
Best: 0.780421 using {'n_estimators': 70}


In [83]:
y_pred_gsgbc = gs_gbc.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gsgbc, gs_gbc)

Accuracy Score:  0.8213333333333334
F1 Score:  0.46719681908548716
ROC-AUC Score:  0.7803374731131514
Recall Score:  0.357958872810358
Precision Score:  0.6723891273247496
PR-AUC Score:  0.5442969677371212


In [85]:
params_2 = {'max_depth': range(5, 16, 2), 'min_samples_split': range(200, 1001, 200)}
gs_gbc_2 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=42, n_estimators=70),
                        param_grid = params_2, 
                        n_jobs = 4,
                        iid = False, 
                        cv = 5, 
                        scoring = 'roc_auc',
                        verbose = 1).fit(X_train_ss, y_train)
print("Best: %f using %s" % (gs_gbc_2.best_score_, gs_gbc_2.best_params_))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:  9.0min finished
Best: 0.781568 using {'max_depth': 5, 'min_samples_split': 1000}


In [86]:
y_pred_gsgbc_2 = gs_gbc_2.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gsgbc_2, gs_gbc_2)

Accuracy Score:  0.8201666666666667
F1 Score:  0.46504709965294994
ROC-AUC Score:  0.7815831606958106
Recall Score:  0.3571972581873572
Precision Score:  0.6661931818181818
PR-AUC Score:  0.5468047052839891


In [90]:
params_3 = {'min_samples_split': range(1000, 2001, 200), 'min_samples_leaf': range(10, 71, 10)}
gs_gbc_3 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=42, n_estimators=70, max_depth=5),
                        param_grid = params_3, 
                        n_jobs = 4,
                        iid = False, 
                        cv = 5, 
                        scoring = 'roc_auc',
                        verbose = 1).fit(X_train_ss, y_train)
print("Best: %f using %s" % (gs_gbc_3.best_score_, gs_gbc_3.best_params_))

Fitting 5 folds for each of 42 candidates, totalling 210 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  7.2min
[Parallel(n_jobs=4)]: Done 210 out of 210 | elapsed:  8.0min finished
Best: 0.783317 using {'min_samples_leaf': 30, 'min_samples_split': 1800}


In [91]:
y_pred_gsgbc_3 = gs_gbc_3.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gsgbc_3, gs_gbc_3)

Accuracy Score:  0.8205
F1 Score:  0.46603867129400095
ROC-AUC Score:  0.7830263610956786
Recall Score:  0.357958872810358
Precision Score:  0.6676136363636364
PR-AUC Score:  0.5485997794238602


In [92]:
params_4 = {'max_features': range(7, 20, 2)}
gs_gbc_4 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=42, n_estimators=70, min_samples_split=1800, min_samples_leaf=30),
                        param_grid = params_4, 
                        n_jobs = 4,
                        iid = False, 
                        cv = 5, 
                        scoring = 'roc_auc',
                        verbose = 1).fit(X_train_ss, y_train)
print("Best: %f using %s" % (gs_gbc_4.best_score_, gs_gbc_4.best_params_))

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:   24.4s finished
Best: 0.781619 using {'max_features': 9}


In [94]:
y_pred_gsgbc_4 = gs_gbc_4.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gsgbc_4, gs_gbc_4)

Accuracy Score:  0.8201666666666667
F1 Score:  0.46504709965294994
ROC-AUC Score:  0.7804782751338107
Recall Score:  0.3571972581873572
Precision Score:  0.6661931818181818
PR-AUC Score:  0.5437970926408667


In [93]:
params_5 = {'subsample': [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]}
gs_gbc_5 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=42, n_estimators=70, min_samples_split=1800, 
                                                               min_samples_leaf=30, max_features=9),
                        param_grid = params_5, 
                        n_jobs = 4,
                        iid = False, 
                        cv = 5, 
                        scoring = 'roc_auc',
                        verbose = 1).fit(X_train_ss, y_train)
print("Best: %f using %s" % (gs_gbc_5.best_score_, gs_gbc_5.best_params_))

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:   13.5s finished
Best: 0.781776 using {'subsample': 0.7}


In [95]:
y_pred_gsgbc_5 = gs_gbc_5.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gsgbc_5, gs_gbc_5)

Accuracy Score:  0.8195
F1 Score:  0.4672897196261682
ROC-AUC Score:  0.7789054036289385
Recall Score:  0.3617669459253618
Precision Score:  0.6597222222222222
PR-AUC Score:  0.5372269413921732


In [96]:
gbc_tuned = GradientBoostingClassifier(random_state=42, n_estimators=140, min_samples_split=1800, 
                                       min_samples_leaf=30, max_features=9, subsample=0.7, learning_rate=0.05).fit(X_train_ss, y_train)
y_pred_gbc_tuned = gbc_tuned.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gbc_tuned, gbc_tuned)

Accuracy Score:  0.8191666666666667
F1 Score:  0.4678764100049043
ROC-AUC Score:  0.7805194676464905
Recall Score:  0.3632901751713633
Precision Score:  0.6570247933884298
PR-AUC Score:  0.5413461845711439


In [97]:
gbc_tuned_2 = GradientBoostingClassifier(random_state=42, n_estimators=800, min_samples_split=1800, 
                                       min_samples_leaf=30, max_features=9, subsample=0.7, learning_rate=0.01).fit(X_train_ss, y_train)
y_pred_gbc_tuned_2 = gbc_tuned_2.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gbc_tuned_2, gbc_tuned_2)

Accuracy Score:  0.82
F1 Score:  0.46745562130177515
ROC-AUC Score:  0.7811059937787119
Recall Score:  0.361005331302361
Precision Score:  0.6629370629370629
PR-AUC Score:  0.5410529462314562


In [98]:
gbc_tuned_3 = GradientBoostingClassifier(random_state=42, n_estimators=1600, min_samples_split=1800, 
                                       min_samples_leaf=30, max_features=9, subsample=0.7, learning_rate=0.005).fit(X_train_ss, y_train)
y_pred_gbc_tuned_3 = gbc_tuned_3.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gbc_tuned_3, gbc_tuned_3)

Accuracy Score:  0.8201666666666667
F1 Score:  0.46925725528775203
ROC-AUC Score:  0.781126386916153
Recall Score:  0.3632901751713633
Precision Score:  0.6625
PR-AUC Score:  0.5417112302681786


In [ ]:
gbc_tuned_4 = GradientBoostingClassifier(random_state=42, n_estimators=1600, min_samples_split=1800, 
                                       min_samples_leaf=30, max_features=9, subsample=0.7, learning_rate=0.005).fit(X_train_ss, y_train)
y_pred_gbc_tuned_4 = gbc_tuned_4.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_gbc_tuned_4, gbc_tuned_4)

In [39]:
pickle_in = open("data/best_model.pickle","rb")
rfcb = pickle.load(pickle_in)

In [40]:
rfcb

RandomForestClassifier(max_depth=8, n_estimators=400)

# Dummy Classifier

In [99]:
dc = DummyClassifier(strategy='most_frequent').fit(X_train_ss, y_train)
y_pred_dc = dc.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_dc, dc)

Accuracy Score:  0.7811666666666667
F1 Score:  0.0
ROC-AUC Score:  0.5
Recall Score:  0.0
Precision Score:  0.0
PR-AUC Score:  0.21883333333333332


# Ensemble Methods

- Bagging and boosting algorithms
- Easy Ensemble Classifier: bag of balanced boosted learners known as EasyEnsemble
- classifier is ensemble of Adaboost learners trained on different balanced bootstrap samples
- balancing achieved by random under-sampling

## Bagging Classifier

Instead of using a single tree, we will check if an ensemble of decsion tree can actually alleviate the issue induced by the class imbalancing. First, we will use a bagging classifier and its counter part which internally uses a random under-sampling to balanced each boostrap sample.

Balancing each bootstrap sample allows to increase significantly the balanced accuracy and the geometric mean.

In [101]:
bc = BaggingClassifier(n_estimators=50, random_state=42).fit(X_train_ss, y_train)
y_pred_bc = bc.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_bc, bc)
print("")
print('Balanced Accuracy: ', balanced_accuracy_score(y_valid, y_pred_bc))
print('Geometric Mean: ', geometric_mean_score(y_valid, y_pred_bc))

Accuracy Score:  0.8118333333333333
F1 Score:  0.457993278924628
ROC-AUC Score:  0.7488929613776726
Recall Score:  0.3632901751713633
Precision Score:  0.6194805194805195
PR-AUC Score:  0.5017609097848447

Balanced Accuracy:  0.650388420207828
Geometric Mean:  0.583592062006858


## Balanced Bagging Classifier

A Bagging classifier with additional balancing.

This implementation of Bagging is similar to the scikit-learn implementation. It includes an additional step to balance the training set at fit time using a given sampler.

This classifier can serves as a basis to implement various methods such as Exactly Balanced Bagging, Roughly Balanced Bagging, Over-Bagging, or SMOTE-Bagging.

In [102]:
bbc = BalancedBaggingClassifier(n_estimators=50, random_state=42).fit(X_train_ss, y_train)
y_pred_bbc = bbc.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_bbc, bbc)
print("")
print('Validation Balanced Accuracy: ', balanced_accuracy_score(y_valid, y_pred_bbc))
print('Validation Geometric Mean: ', geometric_mean_score(y_valid, y_pred_bbc))

Accuracy Score:  0.783
F1 Score:  0.5244704163623083
ROC-AUC Score:  0.7652801066487965
Recall Score:  0.5468392993145469
Precision Score:  0.503859649122807
PR-AUC Score:  0.5141101768863308

Validation Balanced Accuracy:  0.6979982713769235
Validation Geometric Mean:  0.681434187585433


## Balanced Bagging Classifier with Gradient Boosting Classifier

In [103]:
bbc3 = BalancedBaggingClassifier(
        base_estimator=HistGradientBoostingClassifier(random_state=42),
        n_estimators=10,
        random_state=42,
        n_jobs=2).fit(X_train_ss, y_train)
y_pred_bbc3 = bbc3.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_bbc3, bbc3)

Accuracy Score:  0.7618333333333334
F1 Score:  0.5331590983338778
ROC-AUC Score:  0.7816608333627179
Recall Score:  0.6214775323686215
Precision Score:  0.4668192219679634
PR-AUC Score:  0.541759943308943


## Balanced Random Forest Classifier

Random forest is another popular ensemble method and it is usually outperforming bagging. Here, we used a vanilla random forest and its balanced counterpart in which each bootstrap sample is balanced.

Similarly to the previous experiment, the balanced classifier outperform the classifier which learn from imbalanced bootstrap samples. In addition, random forest outsperforms the bagging classifier.

In [104]:
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=42).fit(X_train_ss, y_train)
y_pred_brf = brf.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_brf, brf)

Accuracy Score:  0.7426666666666667
F1 Score:  0.5199004975124378
ROC-AUC Score:  0.7718711524202592
Recall Score:  0.6367098248286367
Precision Score:  0.4393063583815029
PR-AUC Score:  0.5237137210202338


## RUSBoostClassifier

Random under-sampling integrated in the learning of AdaBoost.

During learning, the problem of class balancing is alleviated by random under-sampling the sample at each iteration of the boosting algorithm

In [106]:
rbc = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R', random_state=42).fit(X_train_ss, y_train)
y_pred_rbc = rbc.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_rbc, rbc)

Accuracy Score:  0.7511666666666666
F1 Score:  0.5200900032144005
ROC-AUC Score:  0.7648475446418779
Recall Score:  0.6161462300076161
Precision Score:  0.449944382647386
PR-AUC Score:  0.5211050914562826


## Easy Ensemble Classifier

Bag of balanced boosted learners also known as EasyEnsemble.

This algorithm is known as EasyEnsemble. The classifier is an ensemble of AdaBoost learners trained on different balanced boostrap samples. The balancing is achieved by random under-sampling.

In [107]:
eec = EasyEnsembleClassifier(random_state=42).fit(X_train_ss, y_train) 
y_pred_eec = eec.predict(X_valid_ss)
get_metric(X_valid_ss, y_valid, y_pred_eec, eec)

Accuracy Score:  0.759
F1 Score:  0.5271419228253761
ROC-AUC Score:  0.7765765235826728
Recall Score:  0.6138613861386139
Precision Score:  0.46189111747851
PR-AUC Score:  0.5280535859382768


# Undersampling Methods

## TomekLinks

- Detects Tomek's links, which exists if two samples from different categories are nearest neighbors
- using `sampling_strategy` controls which of the sample will be removed, majority or minority, where `auto` is removing majority class
- setting to `all` removes both samples

In [108]:
tl = TomekLinks()
sampling(X_train_ss, y_train, X_valid_ss, y_valid, tl, gbc)

Training Count:  Counter({0: 15100, 1: 4656})
Accuracy Score:  0.819
F1 Score:  0.4843304843304843
ROC-AUC Score:  0.7827677501137059
Recall Score:  0.38842345773038844
Precision Score:  0.6431273644388399
PR-AUC Score:  0.5453507704238421


In [111]:
X_train_tl, y_train_tl = tl.fit_sample(X_train_ss, y_train)
tlm = gbc.fit(X_train_tl, y_train_tl)
y_pred_tl = tlm.predict(X_valid_ss)

## Edited Nearest Neighbours

- applies nearest-neighbors algorithm and removes samples which do not agree enough with neighborhood or not in the same class as the neighbors
- two selection criteria:  `mode` and `all`, majority vs. all, so that more samples will be excluded with `all`
- `RepeatedEditedNearestNeighbours()` repeat algorithm muliple times resulting in more samples removed
- `AllKNN()` repeats algorithm but increase the number of nearest neighbors at each iteration

In [109]:
enn = EditedNearestNeighbours()
sampling(X_train_ss, y_train, X_valid_ss, y_valid, enn, gbc)

Training Count:  Counter({0: 9974, 1: 4656})
Accuracy Score:  0.791
F1 Score:  0.5399853264856933
ROC-AUC Score:  0.7810130465706137
Recall Score:  0.5605483625285606
Precision Score:  0.5208775654635527
PR-AUC Score:  0.537673190195926


In [112]:
X_train_enn, y_train_enn = enn.fit_sample(X_train_ss, y_train)
ennm = rfcb.fit(X_train_enn, y_train_enn)
y_pred_enn = ennm.predict(X_valid_ss)

## Near Miss

In [136]:
ns = NearMiss(version=1, n_neighbors=3)
sampling(X_train_ss, y_train, X_valid_ss, y_valid, ns, gbc)

Training Count:  Counter({0: 4656, 1: 4656})
Accuracy Score:  0.49316666666666664
F1 Score:  0.39386087303169226
ROC-AUC Score:  0.6087256466533887
Recall Score:  0.7524752475247525
Precision Score:  0.26673866090712745
PR-AUC Score:  0.26422653123571577


In [138]:
X_train_ns, y_train_ns = ns.fit_sample(X_train_ss, y_train)
nsm = gbc.fit(X_train_ns, y_train_ns)
y_pred_ns = nsm.predict(X_valid_ss)

## Neighborhood Cleaning Rule

In [139]:
ncr = NeighbourhoodCleaningRule(n_neighbors=3, threshold_cleaning=0.5)
sampling(X_train_ss, y_train, X_valid_ss, y_valid, ncr, gbc)

Training Count:  Counter({0: 10218, 1: 4656})
Accuracy Score:  0.7936666666666666
F1 Score:  0.5411415863602669
ROC-AUC Score:  0.7785372709367242
Recall Score:  0.555978674790556
Precision Score:  0.5270758122743683
PR-AUC Score:  0.5375883115780334


In [140]:
X_train_ncr, y_train_ncr = ncr.fit_sample(X_train_ss, y_train)
ncrm = gbc.fit(X_train_ncr, y_train_ncr)
y_pred_ncr = ncrm.predict(X_valid_ss)

## One Sided Selection

In [141]:
oss = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
sampling(X_train_ss, y_train, X_valid_ss, y_valid, oss, gbc)

Training Count:  Counter({0: 13843, 1: 4656})
Accuracy Score:  0.8191666666666667
F1 Score:  0.485538169748696
ROC-AUC Score:  0.7795020857061006
Recall Score:  0.38994668697638996
Precision Score:  0.6432160804020101
PR-AUC Score:  0.5436972301524162


In [142]:
X_train_oss, y_train_oss = oss.fit_sample(X_train_ss, y_train)
ossm = gbc.fit(X_train_oss, y_train_oss)
y_pred_oss = ossm.predict(X_valid_ss)

# Other Undersampling Methods

- Methods that select samples to keep
    - Condensed Nearest Neighbors
    - Near Miss
- Methods that select samples to delete
- Methods that use combination of both techniques
    - Neighborhood Cleaning Rule
    - One Sided Selection

# Oversampling Methods

## SMOTE

In [122]:
sm = SMOTE(sampling_strategy='minority', random_state=42)
sampling(X_train_ss, y_train, X_valid_ss, y_valid, sm, gbc)

Training Count:  Counter({0: 16344, 1: 16344})
Accuracy Score:  0.7935
F1 Score:  0.516582130316036
ROC-AUC Score:  0.7640743928654243
Recall Score:  0.5041888804265042
Precision Score:  0.5296
PR-AUC Score:  0.5268566190077079


In [147]:
X_train_sm, y_train_sm = sm.fit_sample(X_train_ss, y_train)
smm = gbc.fit(X_train_sm, y_train_sm)
y_pred_sm = smm.predict(X_valid_ss)

## SVMSMOTE

- Variant of SMOTE algorithm which uses an SVM algorithm to detect sample for generating new synthetic samples

In [114]:
svmsm = SVMSMOTE()
sampling(X_train_ss, y_train, X_valid_ss, y_valid, svmsm, gbc)

Training Count:  Counter({0: 16344, 1: 16344})
Accuracy Score:  0.7995
F1 Score:  0.5287896592244419
ROC-AUC Score:  0.771839790862282
Recall Score:  0.5140898705255141
Precision Score:  0.5443548387096774
PR-AUC Score:  0.5200407308276643


In [146]:
X_train_svm, y_train_svm = svmsm.fit_sample(X_train_ss, y_train)
svmsmote = gbc.fit(X_train_svm, y_train_svm)
y_pred_svmsm = svmsmote.predict(X_valid_ss)

## ADASYN

- Oversample using Adaptive Synthetic (ADASYN) algorithm

In [116]:
adsn = ADASYN()
sampling(X_train_ss, y_train, X_valid_ss, y_valid, adsn, gbc)

Training Count:  Counter({0: 16344, 1: 15923})
Accuracy Score:  0.7905
F1 Score:  0.5099415204678363
ROC-AUC Score:  0.7601081307520225
Recall Score:  0.49809596344249807
Precision Score:  0.5223642172523961
PR-AUC Score:  0.522026068617647


In [145]:
X_train_adsn, y_train_adsn = adsn.fit_sample(X_train_ss, y_train)
adsnm = gbc.fit(X_train_adsn, y_train_adsn)
y_pred_adsnm = adsnm.predict(X_valid_ss)

# Combined Methods

## SMOTE-Tomek

- oversampling using SMOTE and cleaning using Tomek Links

In [118]:
smtk = SMOTETomek()
sampling(X_train_ss, y_train, X_valid_ss, y_valid, smtk, gbc)

Training Count:  Counter({0: 15938, 1: 15938})
Accuracy Score:  0.7951666666666667
F1 Score:  0.5212310089598754
ROC-AUC Score:  0.76603782138894
Recall Score:  0.5095201827875095
Precision Score:  0.5334928229665071
PR-AUC Score:  0.5255432844118494


In [144]:
X_train_smt, y_train_smt = smtk.fit_sample(X_train_ss, y_train)
smtkm = gbc.fit(X_train_smt, y_train_smt)
y_pred_smtk = smtkm.predict(X_valid_ss)

## SMOTE-ENN

- Oversampling using SMOTE and cleaning using ENN

In [119]:
smenn = SMOTEENN(sampling_strategy="minority", n_jobs= -1)
sampling(X_train_ss, y_train, X_valid_ss, y_valid, smenn, gbc)

Training Count:  Counter({1: 12432, 0: 8515})
Accuracy Score:  0.73
F1 Score:  0.5117540687160941
ROC-AUC Score:  0.7706350520496241
Recall Score:  0.6466108149276466
Precision Score:  0.4234413965087282
PR-AUC Score:  0.52638475073663


In [143]:
X_train_sme, y_train_sme = smenn.fit_sample(X_train, y_train)
smennm = gbc.fit(X_train_sme, y_train_sme)
y_pred_smenn = smennm.predict(X_valid_ss)

In [148]:
data = {'Accuracy': [accuracy(y_valid, y_pred_bc),
                     accuracy(y_valid, y_pred_bbc),
                     accuracy(y_valid, y_pred_bbc3),
                     accuracy(y_valid, y_pred_brf),
                     accuracy(y_valid, y_pred_rbc),
                     accuracy(y_valid, y_pred_eec),
                     accuracy(y_valid, y_pred_tl),
                     accuracy(y_valid, y_pred_enn),
                     accuracy(y_valid, y_pred_sm),
                     accuracy(y_valid, y_pred_svmsm),
                     accuracy(y_valid, y_pred_adsnm),
                     accuracy(y_valid, y_pred_smtk),
                     accuracy(y_valid, y_pred_smenn)],
    'F1 Score': [f1_score(y_valid, y_pred_bc),
                 f1_score(y_valid, y_pred_bbc),
                 f1_score(y_valid, y_pred_bbc3),
                 f1_score(y_valid, y_pred_brf),
                 f1_score(y_valid, y_pred_rbc),
                 f1_score(y_valid, y_pred_eec),
                 f1_score(y_valid, y_pred_tl),
                 f1_score(y_valid, y_pred_enn),
                 f1_score(y_valid, y_pred_sm),
                 f1_score(y_valid, y_pred_svmsm),
                 f1_score(y_valid, y_pred_adsnm),
                 f1_score(y_valid, y_pred_smtk),
                 f1_score(y_valid, y_pred_smenn)],
    'ROC-AUC Score': [auc(X_valid_ss, y_valid, bc),
                      auc(X_valid_ss, y_valid, bbc),
                      auc(X_valid_ss, y_valid, bbc3),
                      auc(X_valid_ss, y_valid, brf),
                      auc(X_valid_ss, y_valid, rbc),
                      auc(X_valid_ss, y_valid, eec),
                      auc(X_valid_ss, y_valid, tlm),
                      auc(X_valid_ss, y_valid, ennm),
                      auc(X_valid_ss, y_valid, smm),
                      auc(X_valid_ss, y_valid, svmsmote),
                      auc(X_valid_ss, y_valid, adsnm),
                      auc(X_valid_ss, y_valid, smtkm),
                      auc(X_valid_ss, y_valid, smennm)],
    'PR-AUC Score': [aps(X_valid_ss, y_valid, bc),
                     aps(X_valid_ss, y_valid, bbc),
                     aps(X_valid_ss, y_valid, bbc3),
                     aps(X_valid_ss, y_valid, brf),
                     aps(X_valid_ss, y_valid, rbc),
                     aps(X_valid_ss, y_valid, eec),
                     aps(X_valid_ss, y_valid, tlm),
                     aps(X_valid_ss, y_valid, ennm),
                     aps(X_valid_ss, y_valid, smm),
                     aps(X_valid_ss, y_valid, svmsmote),
                     aps(X_valid_ss, y_valid, adsnm),
                     aps(X_valid_ss, y_valid, smtkm),
                     aps(X_valid_ss, y_valid, smennm)]}
scores = pd.DataFrame(data=data, index = ['Bagging Classifier',
                                          'Balanced Bagging Classifier',
                                          'BBC with Gradient Boosting',
                                          'Balanced Random Forest',
                                          'RUBoost Classifier',
                                          'Easy Ensemble Classifier',
                                          'Tomek Links',
                                          'Edited Nearest Neighbours',
                                          'SMOTE',
                                          'SVM-SMOTE',
                                          'ADASYN',
                                          'SMOTE-Tomek',
                                          'SMOTE-ENN'])

In [149]:
scores

,Accuracy,F1 Score,ROC-AUC Score,PR-AUC Score
Bagging Classifier,0.811833,0.457993,0.748893,0.501761
Balanced Bagging Classifier,0.783000,0.524470,0.765280,0.514110
BBC with Gradient Boosting,0.761833,0.533159,0.781661,0.541760
Balanced Random Forest,0.742667,0.519900,0.771871,0.523714
RUBoost Classifier,0.751167,0.520090,0.764848,0.521105
Easy Ensemble Classifier,0.759000,0.527142,0.776577,0.528054
Tomek Links,0.819000,0.484330,0.764074,0.526857
Edited Nearest Neighbours,0.795833,0.539300,0.443035,0.218539
SMOTE,0.793500,0.516582,0.764074,0.526857
SVM-SMOTE,0.798667,0.531783,0.764074,0.526857
